# Essais de requêtes

## Accession
Aller chercher un numéro d'accession pour l'espèce qui servira d'entrée pour trouver la BdD ensembl appropriée

In [22]:
# !/usr/bin/ python3
#-*- coding : utf-8 -*- 
import requests, sys

## Fonction pour requête API et sortie en Json
def RequestJson (server, ext):
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    
    #Retour en cas d'erreur
    if not r.ok:
        r.raise_for_status()
        sys.exit()

    return(r.json())

## Fonction pour récupérer la division Ensembl en cas d'échec de la méthode principale
def RetrieveDivAlt(species) :

    # Retrouve les divisions ensembl 
    server = "https://rest.ensembl.org"
    ext = "/info/divisions?" 
 
    divisions = RequestJson(server,ext)[::-1] # Inverse pour chercher dans les espèce bactériennes d'abord

    # Retrouve la division associé à l'espèce
    j = 0
    div = None
    while not div and j<len(divisions) :
        
        ext = f"/info/species?division={divisions[j]}"     
        decoded = RequestJson(server,ext)

        for i in range(len(decoded["species"])) :
            if species in decoded["species"][i]["name"]:
                div = decoded["species"][i]["division"]
        j+=1
    return(div)

## Fonction pour récupérer la division Ensembl 
def RetrieveDiv (species) :
    # Requête des infos concernant l'espèce
    server = "https://rest.ensembl.org" # Serveur Ensembl pour requête API
    ext = f"/info/assembly/{species}?" # Requête d'un assemblage de l'espèce
    decoded = RequestJson(server, ext) # Résultat requête

    accession = decoded['assembly_accession'] # Récupération d'une accession d'un génome    

    # Requête des infos concernant l'accession
    ext = f"/info/genomes/assembly/{accession}?"
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"}) 

    # Retour en cas d'erreur
    if not r.ok:
        division = RetrieveDivAlt(species)
    else : 
        division = r.json()["division"] # Résultat API

    if division[7:].lower() == "vertebrates" :
        div = "www"
    else : 
        div = division[7:].lower()
    
    return(div)

## Fonction pour récupérer les informations relatives au symbole dans l'espèce spécifiée
def InfoGene (species, symbol) :
    # Serveur Ensembl pour requête API
    server = "https://rest.ensembl.org"

    # Requête vers l'espèce et symbole de gène courants
    ext = f"/lookup/symbol/{species}/{symbol}?expand=1"     
    decoded = RequestJson(server,ext)
    
    # Construction du dictionnaire contenant les infos
    species_info = {}
    species_info["species"] = species
    species_info["gene_symbol"] = symbol
    species_info["gene_id"] = decoded["id"]
    species_info["division"] = RetrieveDiv(species)

    for i in decoded["Transcript"] :
        if "transcript_id" not in species_info :
            species_info["transcript_id"] = [i["id"]]
            species_info["prot_id"] = [i["Translation"]["id"]]
        else :
            # Formatage pour le HTML 
            species_info["transcript_id"].append(i["id"]) 
            species_info["prot_id"].append(i["Translation"]["id"])

    species_info["gene_browser"] = f"https://{species_info["division"]}.ensembl.org/{species.capitalize()}/Location/View?db=core;g={species_info["gene_id"]}"
    return(species_info)

with open("GeneSymbols_45.txt", "r") as infos:
    for line in infos:
        symbol = line.split(",")[0]
        current_species = line[:-1].split(",")[1]
        print(InfoGene(current_species, symbol))


{'species': 'homo_sapiens', 'gene_symbol': 'RAD51', 'gene_id': 'ENSG00000051180', 'division': 'www', 'transcript_id': ['ENST00000527860', 'ENST00000423169', 'ENST00000557850', 'ENST00000382643', 'ENST00000267868', 'ENST00000645673', 'ENST00000525066', 'ENST00000526763', 'ENST00000532743', 'ENST00000531277', 'ENST00000533741'], 'prot_id': ['ENSP00000432759', 'ENSP00000406602', 'ENSP00000454176', 'ENSP00000372088', 'ENSP00000267868', 'ENSP00000493712', 'ENSP00000431864', 'ENSP00000431897', 'ENSP00000433924', 'ENSP00000436512', 'ENSP00000433053'], 'gene_browser': 'https://www.ensembl.org/Homo_sapiens/Location/View?db=core;g=ENSG00000051180'}
{'species': 'arabidopsis_thaliana', 'gene_symbol': 'PHYB', 'gene_id': 'AT2G18790', 'division': 'plants', 'transcript_id': ['AT2G18790.2', 'AT2G18790.1'], 'prot_id': ['AT2G18790.2', 'AT2G18790.1'], 'gene_browser': 'https://plants.ensembl.org/Arabidopsis_thaliana/Location/View?db=core;g=AT2G18790'}
{'species': 'saccharomyces_cerevisiae', 'gene_symbol': 

## Création HTML
écrire structure et modifier html

In [26]:
dic = {'species': 'homo_sapiens', 'gene_symbol': 'RAD51', 'gene_id': 'ENSG00000051180', 'division': 'www', 'transcript_id': ['ENST00000527860', 'ENST00000423169', 'ENST00000557850', 'ENST00000382643', 'ENST00000267868', 'ENST00000645673', 'ENST00000525066', 'ENST00000526763', 'ENST00000532743', 'ENST00000531277', 'ENST00000533741'], 'prot_id': ['ENSP00000432759', 'ENSP00000406602', 'ENSP00000454176', 'ENSP00000372088', 'ENSP00000267868', 'ENSP00000493712', 'ENSP00000431864', 'ENSP00000431897', 'ENSP00000433924', 'ENSP00000436512', 'ENSP00000433053'], 'gene_browser': 'https://www.ensembl.org/Homo_sapiens/Location/View?db=core;g=ENSG00000051180'}

html = open("EMBL.html","w")

## HTML Head
head_html = """
<!DOCTYPE HTML>
<html lang="fr-FR">
	<head>
		<title> Projet API </title>
		<meta http-equiv="Content-type" content="text/html; charset=utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">

        <!-- CSS -->
        <link type="text/css" href="https://cdn.datatables.net/2.2.1/css/dataTables.dataTables.min.css" rel="stylesheet">
        <link type="text/css" href="https://cdn.datatables.net/buttons/3.2.0/css/buttons.dataTables.min.css" rel="stylesheet">
        <link type="text/css" href="https://cdn.datatables.net/colreorder/2.0.4/css/colReorder.dataTables.min.css" rel="stylesheet">
        <link type="text/css" href="https://cdn.datatables.net/fixedheader/4.0.1/css/fixedHeader.dataTables.min.css" rel="stylesheet">
        <link type="text/css" href="https://cdn.datatables.net/fixedcolumns/5.0.4/css/fixedColumns.dataTables.min.css" rel="stylesheet">
        <link type="text/css" href="https://cdn.datatables.net/responsive/3.0.3/css/responsive.dataTables.min.css" rel="stylesheet">
        <link type="text/css" href="https://cdn.datatables.net/rowreorder/1.5.0/css/rowReorder.dataTables.min.css" rel="stylesheet">

        <!-- JavaScript -->
        <script type="text/javascript" language="javascript" src="https://code.jquery.com/jquery-3.7.0.js"></script> 
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/2.2.1/js/dataTables.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/buttons/3.2.0/js/dataTables.buttons.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/buttons/3.2.0/js/buttons.colVis.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/colreorder/2.0.4/js/dataTables.colReorder.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/fixedcolumns/5.0.4/js/dataTables.fixedColumns.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/fixedheader/4.0.1/js/dataTables.fixedHeader.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/responsive/3.0.3/js/dataTables.responsive.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/rowreorder/1.5.0/js/dataTables.rowReorder.min.js"></script>

        <!-- Script Supplémentaire-->
        <script type="text/javascript" class="init">
            $(document).ready(function() {
                const table = $('#gene').DataTable({
                    layout: {
                        topStart: {
                            buttons: ['colvis']
                        }
                    },
                    fixedHeader: true,
                    "scrollY": 700,
                    "scrollX": true,
                    lengthMenu: [
                        [5, 10, 25, 50, -1],
                        [5, 10, 25, 50, 'All']
                    ],
                    colReorder: {
                        columns: ':gt(1)'
                    },                                       
                });
            });
            </script>
    </head>
    <body>
        <h1>Resultat du scripting d'aggrégation automatique des annotations</h1>

        <table id="gene" class="display nowrap cell-border" style="width:100%">
            <thead>
                <tr>
                    <th><div class=header_1>Species</div></th>
                    <th>Gene Symbol</th>
                    <th>Gene Browser</th>
                    <th>Gene - EMBL ID</th>
                    <th>Protein - EMBL ID</th>
                    <th>Transcript - EMBL ID</th>
                </tr>
            </thead>
            <tbody>
"""

## HTML Body
body_html =""
# Boucle à travers le fichier d'origine pour créer les lignes du tableau
with open("GeneSymbols_45.txt", "r") as infos:
    for line in infos:
        symbol = line.split(",")[0]
        current_species = line[:-1].split(",")[1]
        dic= InfoGene(current_species, symbol)        
        
        link = f"https://{dic['division']}.ensembl.org/{dic['species'].capitalize()}" # Raccourci
        
        # Créer les liens pour les protéines et les transcrits
        
        transcript = dic["transcript_id"][0]
        html_transcript = f"<a href = {link}/Transcript/Summary?db=core;g={dic["species"]};t={transcript}>{transcript}</a>"
        html_prot = f"<a href = {link}/Transcript/ProteinSummary?db=core;g={dic["species"]};t={transcript}>{dic["prot_id"][0]}</a>"

        i=1
        for i in range(len(dic["transcript_id"][1:])) :
            transcript = dic["transcript_id"][i]
            html_transcript += f"<br>\n\t\t\t<a href = {link}/Transcript/Summary?db=core;g={dic["species"]};t={transcript}>{transcript}</a>"
            html_prot += f"<br>\n\t\t\t<a href = {link}/Transcript/ProteinSummary?db=core;g={dic["species"]};t={transcript}>{dic["prot_id"][i]}</a>"

        body_html += f"""
                <tr>                    
                    <td><div class=header_1>
                        {dic['species']}
                    </div></td>
                    <td>
                        {dic["gene_symbol"]}
                    </td>
                    <td><a href = {dic["gene_browser"]}>
                        View {dic["gene_symbol"]} in gene browser
                    </a></td>
                    <td><a href = {link}/Gene/Summary?db=core;g={dic['gene_id']}>
                        {dic["gene_id"]}
                    </a></td>
                    <td>
                        {html_transcript}
                    </td>
                    <td>
                        {html_prot}
                    </td>
                </tr>"""

tail_html = f""" 
            </tbody>
        </table>
    </body>
</html>
"""

html = open("EMBL.html","w")
html.write(head_html[1:]+body_html+tail_html)
html.close()


## Sauvegarde
Y'a des bouts de code utiles  
```python  
decoded = (json.dumps(decoded,indent=2))
species_info["transcript_id"] += "<br>\n\t\t\t" + i["id"]
species_info["prot_id"] += "<br>\n\t\t\t" + i["Translation"]["id"]
```

In [15]:
dic = {'species': 'homo_sapiens', 'gene_symbol': 'RAD51', 'gene_id': 'ENSG00000051180', 'division': 'www', 'transcript_id': ['ENST00000527860', 'ENST00000423169', 'ENST00000557850', 'ENST00000382643', 'ENST00000267868', 'ENST00000645673', 'ENST00000525066', 'ENST00000526763', 'ENST00000532743', 'ENST00000531277', 'ENST00000533741'], 'prot_id': ['ENSP00000432759', 'ENSP00000406602', 'ENSP00000454176', 'ENSP00000372088', 'ENSP00000267868', 'ENSP00000493712', 'ENSP00000431864', 'ENSP00000431897', 'ENSP00000433924', 'ENSP00000436512', 'ENSP00000433053'], 'gene_browser': 'https://www.ensembl.org/Homo_sapiens/Location/View?db=core;g=ENSG00000051180'}
#https://www.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000051180;r=15:40694774-40718886;t=ENST00000527860
link = f"https://{dic["division"]}.ensembl.org/{dic["species"].capitalize()}"
print (link)

https://www.ensembl.org/Homo_sapiens
